In [214]:
import json

import numpy as np
import pandas as pd
import pymongo
import scipy
from bson import ObjectId, json_util
from pandas.io.json import json_normalize
from pymongo import MongoClient as Connection
from scipy import sparse, spatial
from scipy.spatial.distance import cdist, pdist, squareform
from sklearn.metrics.pairwise import cosine_similarity

# scientific notation disabled form smaller numbers
pd.options.display.float_format = '{:.5f}'.format

# alles resultate anzeigen und nicht nur das letzte
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"



In [215]:
# display multiple outputs in one row
import pandas as pd
import numpy as np
from IPython.display import display, HTML

CSS = """
.output {
    flex-direction: row;
}
"""

HTML('<style>{}</style>'.format(CSS))

In [216]:
connection = Connection()
db = connection.recipe_db
input_data = db.recipes_test_copy

data = json.loads(json_util.dumps(input_data.find()))

In [217]:
ing = pd.DataFrame(json_normalize(data, record_path='ingredients',
                             meta='id', record_prefix='ingredients_', errors='ignore'))


nutritions = pd.DataFrame(json_normalize(data, record_path='nutritions',
                            meta=['id', 'prep_time', 'rating', 'rating_count', 'ready_in_time', 'review_count']))


In [218]:
#  ------  erstellung und data cleansing - Ingredients

# schmeiss alle zutaten raus, die weniger als 5 mal verwendet werden
# setze multiinde auf 'id' und 'ingredients_id'
ingredients = ing.set_index(['id', 'ingredients_id'])

# filtere alle Zutaten samt ihrer rezepte id, die weniger gleich 5 mal vorkommen
ingredients_eqles_5_ing = ingredients.groupby(
    'ingredients_id').filter(lambda x: len(x) <= 5)

# droppe alle rezepte, die eine Zutate besitzen, die weniger gleich 5 Mal vorkommen
ingredients_filt = ingredients.drop(ingredients_eqles_5_ing.index.get_level_values('id').values, level=0)

# drop alls rows with ingredients_id == 0
ingredients_eqal_zero = ingredients_filt[ingredients_filt.index.get_level_values('ingredients_id') == 0]
ingredients_filt = ingredients_filt.drop(ingredients_eqal_zero.index.get_level_values('id').values, level=0)


In [219]:
#  ------ Erstellung und cleansing des Nutrition Dataframes

# erstelle neue liste auf basis der bereits gefilterten rezepte aus ingredients_filt
id_overlap_mask = nutritions['id'].isin(ingredients_filt.index.get_level_values('id').values)

# erstelle datenframe auf basis der overlapliste
nutritions_filt = nutritions.loc[id_overlap_mask]

nutrition_db = nutritions_filt.pivot_table(
    index=['id'],
    columns=['name'],
    values=['amount'],
).reset_index()

nutrition_db.set_index('id', inplace=True)

# remove multiindex 'amount'
nutrition_db.columns = nutrition_db.columns.droplevel(0)

# entferne alle NA
nutrition_db = nutrition_db.dropna()

# gleiche nochmals die ids der beiden dataframe nutrition und ingredients ab, da der nutrition dataframe noch NA Werte hatt
id_overlap_mask = ingredients_filt.index.get_level_values('id').isin(nutrition_db.index)
ingredients_db = ingredients_filt[id_overlap_mask]

# abgleich ob anzahl der indizes von nutrition und zutaten dataframe gleich sind


In [220]:


ingredients_db.reset_index(inplace=True)

recipe_db = pd.get_dummies(ingredients_db['ingredients_id']).groupby(
    ingredients_db['id']).apply(max)



In [221]:
new_ingredients = ingredients_db.copy()
#new_ingredients = new_ingredients.groupby("id")["ingredients_grams"].sum().reset_index()
gramms_ingredients = new_ingredients.groupby("id")["ingredients_grams"].sum().reset_index().copy()

Q1 = gramms_ingredients.quantile(0.25)
Q3 = gramms_ingredients.quantile(0.75)
IQR = Q3 - Q1

#Filter out all recipes which are outlier by their weight (gramms)
df = gramms_ingredients[~((gramms_ingredients < (Q1 - 1.5 * IQR)) |(gramms_ingredients > (Q3 + 1.5 * IQR))).any(axis=1)].copy()

#filter out recipes by weight which are not in the range 500 - 2373.59 gramms
df_start_at_fivehundret = df[df['ingredients_grams'].between(500, 2373.58225, inclusive=False)].copy()

df_start_at_fivehundret.set_index('id', inplace=True)
id_overlap_mask = nutritions['id'].isin(df_start_at_fivehundret.index.get_level_values('id').values)


# erstelle datenframe auf basis der overlapliste
nutritions_filt_gramm = nutritions.loc[id_overlap_mask]

nutrition_db2 = nutritions_filt_gramm.pivot_table(
    index=['id'],
    columns=['name'],
    values=['amount'],
).reset_index()

#create new nutrition db based on the above filtering
nutrition_db2.set_index('id', inplace=True)
nutrition_db2.columns = nutrition_db2.columns.droplevel(0)



In [222]:
#select 20 random recipes which are between the Fat-range of 35-40 gramms
subset_fat = nutrition_db2[(nutrition_db2['Fat'] >= 35) & (nutrition_db2['Fat'] <= 40)].sample(n=20).copy()

subset_fat_id = subset_fat.index.to_numpy()


In [223]:
subset_normal = nutrition_db2.sample(n=20).copy()
subset_normal_id = subset_normal.index.to_numpy()

In [224]:
subset_normal.index.to_numpy()

array([ 70680,  14772,  68539, 217984, 222582,  46859, 165190,   8731,
        27838,  54342,  20059,  15486,  12045,  21360, 164957,  12017,
        26637, 105553,  83823, 221509])

In [225]:
# remove recipes which are missing in nutrition df from the ingredients tecipes dataframe
mask = recipe_db.index.isin(nutrition_db2.index.values)
recipe_db = recipe_db.loc[mask]


In [226]:
user_recipes_fat = recipe_db[recipe_db.index.isin(subset_fat_id)]
user_recipes_normal = recipe_db[recipe_db.index.isin(subset_normal_id)]

In [227]:
#top10 most common ingredients
#16421    2125 salt
#4342     1506 garlic
#4397     1412 onion
#16406    1185 ground black pepper
#16157    1016 butter
#6307      944 olive oil
#6494      786 skinless bone less chicken breast
#2496      749 water
#16238     574 grated Parmesan cheese
#16317     538 eggs


drop_id_list = [16421, 4342, 4397, 16406, 16157, 6307, 6494, 2496, 16238, 16317]





In [228]:
subset_fat_id

array([151997,  13420,  50849, 202881,   7198,  69538,  51653,  20644,
       161869,  52501,  17496,  36766,  26299,  23985,  73331, 235171,
         8761,  48873,  19919, 221294])

In [229]:
subset_normal_id

array([ 70680,  14772,  68539, 217984, 222582,  46859, 165190,   8731,
        27838,  54342,  20059,  15486,  12045,  21360, 164957,  12017,
        26637, 105553,  83823, 221509])

In [230]:
#filter out ingredients which are too common and the recipes in the user profile from the recipes database
new_recipe_db = recipe_db.drop(axis=1, labels=drop_id_list)
new_recipe_db_wo_userrecipes_fat = new_recipe_db.drop(axis=0, labels=subset_fat_id)
new_recipe_db_wo_userrecipes_normal = new_recipe_db.drop(axis=0, labels=subset_normal_id)

In [231]:
#remove basic ingredients like garlic, olive oil, skinless bone less chicken breast, butter, milk, salt and pepper, onion
new_user_recipes_fat = user_recipes_fat.drop(axis=1, labels=drop_id_list)
new_user_recipes_normal = user_recipes_normal.drop(axis=1, labels=drop_id_list)

In [232]:
asd = new_user_recipes_fat[0:10]

In [233]:
asd

,111,126,257,443,445,578,615,629,631,858,...,23047,23274,23383,24865,25518,25522,26269,26934,27343,27805
id,,,,,,,,,,,,,,,,,,,,,
7198,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
8761,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
13420,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
17496,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
19919,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
20644,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
23985,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
26299,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
36766,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [250]:
new_user_recipes_fat[0:20]

,111,126,257,443,445,578,615,629,631,858,...,23047,23274,23383,24865,25518,25522,26269,26934,27343,27805
id,,,,,,,,,,,,,,,,,,,,,
7198,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
8761,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
13420,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
17496,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
19919,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
20644,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
23985,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
26299,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
36766,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [234]:
## Jaccard - recipe to recipe similarity with removal of basis ingredients and duplicate userrecipes

result_array = cdist(new_user_recipes_fat[0:10], new_recipe_db_wo_userrecipes_fat,'jaccard')
result_w_filter = pd.DataFrame(result_array, columns=new_recipe_db_wo_userrecipes_fat.index.values, index=new_user_recipes_fat[0:10].index.values)
# result_2

result_w_filter_10 = pd.DataFrame(result_w_filter.sum(), columns=['jaccard_distance_sum'])
result_w_filter_10 = result_w_filter_10.sort_values(by='jaccard_distance_sum')
result_w_filter_10['jaccard_distance_sum'] = result_w_filter_10['jaccard_distance_sum'].div(10)
result_w_filter_10[0:10]


,jaccard_distance_sum
195045,0.87222
216231,0.87750
87624,0.87995
234397,0.88500
222582,0.89000
18442,0.89154
214369,0.89603
12073,0.90195
23181,0.90413
22702,0.90575


In [235]:
result_w_filter[52608]

7198    0.92308
8761    1.00000
13420   0.87500
17496   0.92308
19919   1.00000
20644   1.00000
23985   1.00000
26299   1.00000
36766   0.94118
48873   1.00000
Name: 52608, dtype: float64

In [236]:
result_w_filter_10[0:10].index.to_numpy()

array([195045, 216231,  87624, 234397, 222582,  18442, 214369,  12073,
        23181,  22702])

In [237]:
result_w_filter_test = result_w_filter.copy()

In [238]:
result_w_filter_test

,8493,8494,8495,8496,8497,8498,8500,8503,8506,8509,...,254874,254940,255038,255263,255545,255936,257312,260193,261124,263813
7198,1.00000,1.00000,1.00000,1.00000,1.00000,1.00000,0.83333,1.00000,1.00000,1.00000,...,1.00000,1.00000,1.00000,1.00000,1.00000,1.00000,1.00000,1.00000,1.00000,0.93333
8761,1.00000,1.00000,1.00000,1.00000,1.00000,0.85714,1.00000,1.00000,0.90000,1.00000,...,1.00000,1.00000,0.92857,0.87500,0.92308,1.00000,1.00000,1.00000,1.00000,0.90909
13420,1.00000,0.91667,1.00000,0.75000,1.00000,1.00000,0.87500,1.00000,1.00000,0.80000,...,1.00000,1.00000,1.00000,0.85714,1.00000,1.00000,1.00000,1.00000,0.87500,0.90000
17496,1.00000,1.00000,1.00000,1.00000,0.90909,0.80000,0.92308,1.00000,1.00000,1.00000,...,1.00000,0.90909,0.81250,0.91667,0.87500,1.00000,0.91667,1.00000,1.00000,0.93333
19919,1.00000,1.00000,1.00000,1.00000,1.00000,1.00000,1.00000,1.00000,1.00000,1.00000,...,0.87500,1.00000,1.00000,1.00000,1.00000,1.00000,1.00000,1.00000,1.00000,1.00000
20644,0.87500,1.00000,1.00000,1.00000,1.00000,1.00000,1.00000,1.00000,1.00000,0.85714,...,1.00000,1.00000,1.00000,1.00000,1.00000,1.00000,1.00000,1.00000,1.00000,0.91667
23985,0.85714,1.00000,1.00000,1.00000,1.00000,1.00000,1.00000,1.00000,1.00000,1.00000,...,1.00000,1.00000,1.00000,1.00000,1.00000,1.00000,1.00000,1.00000,1.00000,1.00000
26299,0.87500,1.00000,1.00000,1.00000,1.00000,1.00000,1.00000,1.00000,1.00000,1.00000,...,1.00000,1.00000,1.00000,1.00000,1.00000,1.00000,1.00000,1.00000,1.00000,1.00000
36766,1.00000,1.00000,1.00000,1.00000,1.00000,1.00000,1.00000,1.00000,0.94444,0.92857,...,1.00000,1.00000,0.85000,1.00000,0.90000,1.00000,0.93750,1.00000,1.00000,1.00000
48873,1.00000,1.00000,1.00000,1.00000,1.00000,1.00000,1.00000,1.00000,1.00000,1.00000,...,1.00000,1.00000,0.91667,1.00000,1.00000,1.00000,1.00000,1.00000,1.00000,1.00000


In [239]:
df1 = pd.DataFrame(np.sort(result_w_filter_test.values, axis=0), index=result_w_filter_test.index, columns=result_w_filter_test.columns)

In [240]:
new_columns = df1.columns[df1.ix[df1.first_valid_index()].argsort()]

/usr/local/lib/python3.7/site-packages/ipykernel_launcher.py:1: FutureWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#ix-indexer-is-deprecated
  """Entry point for launching an IPython kernel.


In [241]:
test = df1[new_columns]

In [242]:
test = test.reset_index()

In [243]:
test

,index,34972,195045,87624,216231,222582,8717,234397,8593,8836,...,30578,17125,176132,176359,177497,17105,178498,17035,30490,150156
0,7198,0.28571,0.33333,0.33333,0.40000,0.42857,0.50000,0.50000,0.50000,0.50000,...,1.00000,1.00000,1.00000,1.00000,1.00000,1.00000,1.00000,1.00000,1.00000,1.00000
1,8761,0.87500,0.50000,0.62500,0.50000,0.57143,0.90000,0.70000,1.00000,0.90000,...,1.00000,1.00000,1.00000,1.00000,1.00000,1.00000,1.00000,1.00000,1.00000,1.00000
2,13420,1.00000,0.88889,0.90000,0.87500,0.90000,1.00000,0.80000,1.00000,1.00000,...,1.00000,1.00000,1.00000,1.00000,1.00000,1.00000,1.00000,1.00000,1.00000,1.00000
3,17496,1.00000,1.00000,0.94118,1.00000,1.00000,1.00000,0.91667,1.00000,1.00000,...,1.00000,1.00000,1.00000,1.00000,1.00000,1.00000,1.00000,1.00000,1.00000,1.00000
4,19919,1.00000,1.00000,1.00000,1.00000,1.00000,1.00000,0.93333,1.00000,1.00000,...,1.00000,1.00000,1.00000,1.00000,1.00000,1.00000,1.00000,1.00000,1.00000,1.00000
5,20644,1.00000,1.00000,1.00000,1.00000,1.00000,1.00000,1.00000,1.00000,1.00000,...,1.00000,1.00000,1.00000,1.00000,1.00000,1.00000,1.00000,1.00000,1.00000,1.00000
6,23985,1.00000,1.00000,1.00000,1.00000,1.00000,1.00000,1.00000,1.00000,1.00000,...,1.00000,1.00000,1.00000,1.00000,1.00000,1.00000,1.00000,1.00000,1.00000,1.00000
7,26299,1.00000,1.00000,1.00000,1.00000,1.00000,1.00000,1.00000,1.00000,1.00000,...,1.00000,1.00000,1.00000,1.00000,1.00000,1.00000,1.00000,1.00000,1.00000,1.00000
8,36766,1.00000,1.00000,1.00000,1.00000,1.00000,1.00000,1.00000,1.00000,1.00000,...,1.00000,1.00000,1.00000,1.00000,1.00000,1.00000,1.00000,1.00000,1.00000,1.00000
9,48873,1.00000,1.00000,1.00000,1.00000,1.00000,1.00000,1.00000,1.00000,1.00000,...,1.00000,1.00000,1.00000,1.00000,1.00000,1.00000,1.00000,1.00000,1.00000,1.00000


In [244]:
asd[0:20]

,111,126,257,443,445,578,615,629,631,858,...,23047,23274,23383,24865,25518,25522,26269,26934,27343,27805
id,,,,,,,,,,,,,,,,,,,,,
7198,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
8761,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
13420,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
17496,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
19919,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
20644,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
23985,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
26299,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
36766,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [245]:
asd = test.loc[0].to_frame()

In [246]:
asd[0:11].index.to_numpy()


array(['index', 34972, 195045, 87624, 216231, 222582, 8717, 234397, 8593,
       8836, 214369], dtype=object)

In [247]:
subset_normal_id[0:10]

array([ 70680,  14772,  68539, 217984, 222582,  46859, 165190,   8731,
        27838,  54342])

In [248]:
subset_fat_id

array([151997,  13420,  50849, 202881,   7198,  69538,  51653,  20644,
       161869,  52501,  17496,  36766,  26299,  23985,  73331, 235171,
         8761,  48873,  19919, 221294])